In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Performance of each platform

In [2]:
# graphalytics inserts
data_ins = common.import_gfe("view_graphalytics_inserts").query("omp_threads.isnull() and cluster == 'stones2' and num_threads_read == 0").copy() # data from the experiments
data_ins["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by

# graphalytics load
data_load = common.import_gfe("view_graphalytics_load").query("omp_threads.isnull() and cluster == 'stones2' and num_threads_read == 0").copy()
# shape the dataframe as data_ins
data_load["build_frequency"] = pd.Timedelta(0)
data_load["build_frequency_secs"] = np.NaN
data_load["num_snapshots_created"] = 0
data_load["num_threads_write"] = 0

data = pd.concat([data_ins, data_load])
data["compiler"] = data["compiler_family"].fillna("n/a")

OperationalError: (sqlite3.OperationalError) no such table: view_graphalytics_inserts
[SQL: SELECT * FROM view_graphalytics_inserts]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

Integrate with the results of GraphMat and GraphBlas

In [ ]:
graphmat_weighted = common.import_graphmat("data/graphmat/stones201_weighted.csv").copy()
graphmat_weighted["t_processing"] = graphmat_weighted["t_processing"].apply(lambda x: x.total_seconds())
graphmat_weighted = graphmat_weighted.groupby(["graph", "algorithm"]).agg(completion_time=("t_processing", "median"), count=("t_processing", "count"))
graphmat_weighted = graphmat_weighted.unstack(1)["completion_time"]
graphmat_unweighted = common.import_graphmat("data/graphmat/stones201_unweighted.csv").copy()
graphmat_unweighted["t_processing"] = graphmat_unweighted["t_processing"].apply(lambda x: x.total_seconds())
graphmat_unweighted = graphmat_unweighted.groupby(["graph", "algorithm"]).agg(completion_time=("t_processing", "median"), count=("t_processing", "count"))
graphmat_unweighted = graphmat_unweighted.unstack(1)["completion_time"]
graphmat = pd.concat([
    graphmat_unweighted.loc[["graph500-24", "uniform-24"], ["BFS", "CDLP", "LCC", "PR", "WCC"]], 
    graphmat_weighted.loc[["graph500-24", "uniform-24"], ["SSSP"]]], 
axis=1)
graphmat = graphmat.rename({"BFS": "bfs", "CDLP": "cdlp", "LCC": "lcc", "PR": "pagerank", "SSSP": "sssp", "WCC": "wcc"}, axis=1)
graphmat["library"] = "graphmat"
graphmat["compiler"] = "n/a"
graphmat["build_frequency"] = pd.Timedelta(0)
graphmat["num_threads_read"] = graphmat["num_threads_write"] = 0
graphmat.set_index(["library", "compiler", "build_frequency", "num_threads_read", "num_threads_write"], append=True, inplace=True)
graphmat = graphmat.reorder_levels([1,2,0,3,4,5])
graphblas = common.import_graphmat("data/graphblas/stones216.csv").copy() # same format of graphmat
graphblas["t_processing"] = graphblas["t_processing"].apply(lambda x: x.total_seconds())
graphblas = graphblas.groupby(["graph", "algorithm"]).agg(completion_time=("t_processing", "median"), count=("t_processing", "count"))
graphblas = graphblas.unstack(1)["completion_time"]
graphblas.rename({"BFS": "bfs", "CDLP": "cdlp", "LCC": "lcc", "PR": "pagerank", "SSSP": "sssp", "WCC": "wcc"}, axis=1, inplace=True)
graphblas["library"] = "graphblas"
graphblas["compiler"] = "n/a"
graphblas["build_frequency"] = pd.Timedelta(0)
graphblas["num_threads_read"] = graphblas["num_threads_write"] = 0
graphblas.set_index(["library", "compiler", "build_frequency", "num_threads_read", "num_threads_write"], append=True, inplace=True)
graphblas = graphblas.reorder_levels([1,2,0,3,4,5])

In [ ]:
analysis = data.groupby(["library", "compiler", "graph", "build_frequency", "num_threads_read", "num_threads_write", "algorithm"]) \
    .agg(completion_time=("median_secs", "median"), count=("median_secs", "count"))
analysis = pd.concat( [analysis.unstack("algorithm")[("completion_time")], graphblas, graphmat ] )

In [ ]:
analysis

If the compiler is null, these are the old results built with Clang v10.

# Results per platform

In [ ]:
csr = analysis.loc[("csr-lcc", "gcc", slice(None))].droplevel([1,2,3])
graphmat = analysis.loc[("graphmat"), :].droplevel([0,2,3,4])
graphblas = analysis.loc[("graphblas"), :].droplevel([0,2,3,4])
graphone = analysis.loc[("g1_v4-ref-ignore-build")].droplevel([0,2,3,4])
livegraph = analysis.loc[("livegraph_ro", "gcc", slice(None), slice(None), slice(None), 20)].droplevel([1,2])
llama = analysis.loc[("llama6-ref", "gcc", slice(None), "00:00:10", 0, 16), :].droplevel([0,1,3,4,5]) # dynamic scheduling in OpenMP
stinger = analysis.loc[("stinger-ref"), :].droplevel([1,2,3])
teseo_dense_vertices = analysis.loc[("teseo-ta-lcc-dv.9", "gcc", slice(None), slice(None), slice(None), 40)].droplevel([1,2])
teseo_logical_vertices = analysis.loc[("teseo-ta-lcc.9", "gcc", slice(None), slice(None), slice(None), 40)].droplevel([1,2])

In [ ]:
teseo_logical_vertices

Speed-up of the CSR w.r.t. GraphMat and GraphBlas:

In [ ]:
graphmat/csr.loc[["graph500-24", "uniform-24"]]

In [ ]:
graphblas/csr